In [ ]:

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional,  Dense, Dropout , BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
new_data = pd.read_csv("test.csv")
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
category_mapping = {'Politics': 0, 'Sports': 1, 'Media': 2, 'Market & Economy': 3, 'STEM': 4}
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r'http\S+|www\S+|@\S+|#\S+', '', text)
    text = "".join([char for char in text if char.isalpha() or char.isspace()])


    tokens = word_tokenize(text.lower())

    tokens = [word for word in tokens if word not in stop_words]

    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)
new_data['Discussion'] = new_data['Discussion'].apply(preprocess_text)
new_data = new_data.dropna()
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(new_data['Discussion']).toarray()

In [18]:
model = load_model('tfidf_lstm_model.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

new_data = pd.read_csv("test.csv")

new_data['Discussion'] = new_data['Discussion'].apply(preprocess_text)
new_data = new_data.dropna()
X_new = tfidf_vectorizer.transform(new_data['Discussion']).toarray()

X_new = X_new.reshape(X_new.shape[0], 1, X_new.shape[1])

predictions = model.predict(X_new)
predicted_classes = np.argmax(predictions, axis=1)

result = pd.DataFrame({
    "SampleID": new_data["SampleID"].values,
    "Category": predicted_classes
})

result.head(5)
result.to_csv('output.csv', index=False)

print("Predictions saved to 'output.csv'")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Predictions saved to 'output.csv'
